In [ ]:
from enum import Enum

import instructor
import pandas as pd
from pydantic import Field

from cuery import Prompt, Response, Task, pprint
from cuery.utils import set_env

set_env(apify_secrets=False)

# Create a prompt from simple string
The `Prompt` class expects a list of (jinja) messages with their roles. But it can also be instantiated from a simple string.

In [ ]:
t = "Hello {{name}}! {% for item in ingredients %} {{ item }} {% endfor %}"
p = Prompt.from_string(t)
pprint(p)

# Simplified client/model creation

In [ ]:
# Docstring descriptions will be passed via the response model to the LLM
class Recipe(Response):
    ingredients: list[str]
    """A list of ingredients for the dish."""


prompt = Prompt.from_string("Generate a list of recipe ingredients to make '{{dish}}'.")
task = Task(prompt=prompt, response=Recipe)
pprint(task)

model = "openai/gpt-4.1-mini"  # or e.g. "perplexity/sonar"
responses = await task(context=[{"dish": "spaghetti carbonara"}], model=model)
# responses.to_pandas(explode=False)

In [ ]:
responses.to_pandas(explode=False)

# Inspect LLM queries (containing final prompt send to LLM)

Only available (for now), when multiple rows were processed!

Each task maintains a log of errors and the queries to the LLM provider. Note that the structure of what's sent to the provider may be different for each.

In [ ]:
task.queries[0]

# Choices (enum)
Require LLM to respond with one of N _options_ (fixed categories).

In [ ]:
class Role(Enum):
    PRINCIPAL = "PRINCIPAL"
    TEACHER = "TEACHER"
    STUDENT = "STUDENT"
    OTHER = "OTHER"


class UserDetail(Response):
    age: int
    name: str
    role: Role = Field(description="Correctly assign one of the predefined roles to the user.")


prompt = Prompt.from_string("Please a create a synthetic user profile with age, name and role.")
task = Task(prompt=prompt, response=UserDetail)

response = await task(model="openai/gpt-3.5-turbo")
print(response)
response.to_pandas()

Or using the Literal type

In [ ]:
from typing import Literal


class UserDetail(Response):
    age: int
    name: str
    role: Literal["PRINCIPAL", "TEACHER", "STUDENT", "OTHER"]
    """Correctly assign one of the predefined roles to the user."""


response = await Task(prompt=prompt, response=UserDetail)(model="openai/gpt-3.5-turbo")
response.to_pandas()


# Simple Multivalued fields
Require LLM to respond with a _list_ of values (unconstrained).

In [ ]:
class Ingredients(Response):
    items: list[str] = Field(description="List of ingredients for the recipe.")


prompt = Prompt.from_string("List the ingredients for the following dish: {{dish}}.")
context = [{"dish": "pasta bolognese"}, {"dish": "chocolate cake"}]

task = Task(prompt=prompt, response=Ingredients)
responses = await task(context=context)
print(responses)

In [ ]:
# Maintain the original structure of the responses
responses.to_pandas(explode=False)

In [ ]:
# Explode the list of ingredients into separate rows
responses.to_pandas(explode=True)

In [ ]:
# Convert to simple python records
responses.to_records(explode=False)

# Nested models
Define a more complicated output structure by referencing another response model. 

In this case a list of certain length containing instances of pre-defined response model.

In [ ]:
class Sector(Response):
    sector: str = Field(
        description="Human-readable title(!) of the industrical sector (in NAICS taxonomy)",
        min_length=10,
        max_length=150,
    )
    subsector: str = Field(
        description="Human-readable title(!) of the industrial SUBsector (in NAICS taxonomy)",
        min_length=5,
        max_length=150,
    )
    sector_automation_potential: int = Field(
        description="A score from 1 to 10 indicating the sector's potential for automation",
        ge=0,
        le=10,
    )


class Sectors(Response):
    sectors: list[Sector] = Field(
        description="A list of 1 to 5 NAIC industrial sectors with their AI automation potential",
        min_length=1,
        max_length=5,
    )


sectors_prompt = Prompt.from_string(
    "List some industrial sector in the country of {{country}} that have great AI automation potential."
)

context = [{"country": "Germany"}, {"country": "United States"}, {"country": "Japan"}]
sectors_task = Task(prompt=sectors_prompt, response=Sectors)
responses = await sectors_task(context=context)

In [ ]:
responses.to_pandas(explode=True)

# Chain tasks together
Run multiple tasks one after the other, collecting the results in a single DataFrame.

Keep in mind here that the names of inputs of one task must be the same as the names of outputs in the previous one.

Here we extract first some industrial sectors for each input country, and then some job roles within each sector.

In [ ]:
# Re-uses "sectors" task from previous code cell (!)

from cuery import Chain


class Job(Response):
    job_role: str
    """Name of the job role (job title, less than 50 characters)"""
    job_description: str
    """A short description of the job role (less than 200 characters)"""
    job_automation_potential: int = Field(
        description="A score from 1 to 10 indicating the job's potential for automation",
        ge=0,
        le=10,
    )


class Jobs(Response):
    jobs: list[Job]
    """A list of jobs with their AI automation potential and reasons for that potential"""


jobs_prompt = Prompt.from_string(
    "List some job roles with great AI automation potential in the country of {{country}} and the sector '{{sector}}'"
)

context = pd.DataFrame(
    {
        "country": ["Germany", "United States", "Japan"],
        "PIB": [4.0, 5.0, 3.5],
    }
)

jobs_task = Task(prompt=jobs_prompt, response=Jobs)
chain = Chain(sectors_task, jobs_task)
responses = await chain(context=context)

In [ ]:
responses

# Tools

`Tools` are another thin level of abstraction to make `Tasks` configurable with a clear input interface (the output is already defined by a `Response` model). They're mostly useful with tasks (prompts and response models) that can be customized, i.e. which have configurable parameters that don't depend on the context of the data over which it will be iterated.

We use pydantic again to define the interface. This has the advantage that we can re-use a tool's interface directly for FastAPI endpoints, and therefore also directly as an MCP interface.

In [ ]:
from typing import ClassVar

from cuery.cli import set_env_vars
from cuery.prompt import Prompt
from cuery.response import Response, ResponseClass
from cuery.tool import Tool

set_env_vars(apify_secrets=False)


class Jokes(Response):
    jokes: list[str]


class Joker(Tool):
    n_jokes: int
    topics: list[str]

    response_model: ClassVar[ResponseClass] = Jokes

    @property
    def prompt(self):
        # ${vars} will be substituted once initially. and so values will be constant when iterating over data
        # Jinja variables (and other Jinja syntax) will be evaluated for each request/row/context item
        instructions = "Create ${n_jokes} one-liners about {{topic}}."
        return Prompt(messages=instructions).substitute(n_jokes=self.n_jokes)

    @property
    def context(self):
        return [{"topic": topic} for topic in self.topics]


joker = Joker(n_jokes=3, topics=["cats", "nerds", "youths"])
result = await joker(n_concurrent=10)
result

In [ ]:
result.to_records(explode=False)

In [ ]:
joker.task.queries[0]

In [ ]:
from cuery.cli import set_env_vars
from cuery.tools.flex import generic

set_env_vars(apify_secrets=False)

p = "I need a schema for users having a name and email"
t = generic.SchemaGenerator(instructions=p, model="openai/gpt-4.1")
r = await t()

In [ ]:
from cuery import pprint

pprint(r.to_dict())

# Web search

In [ ]:
import instructor

from cuery import Field, Prompt, Response, Task, pprint


class Citation(Response):
    id: int
    url: str


class Place(Response):
    name: str = Field(..., description="Name of the restaurant.")
    address: str = Field(..., description="Address of the restaurant.")
    telephone: str = Field(..., description="Telephone number of the restaurant.")


class Places(Response):
    summary: str
    citations: list[Citation]


client = instructor.from_provider(
    "openai/gpt-4.1-mini",
    mode=instructor.Mode.RESPONSES_TOOLS_WITH_INBUILT_TOOLS,
    async_client=True,
)

response, completion = await client.responses.create_with_completion(
    input="What are some of the best places to eat Paella in Madrid, Spain? Return a list of restaurants with their name, address and telephone number.",
    tools=[
        {
            "type": "web_search_preview",
            "search_context_size": "low",
            "user_location": {
                "type": "approximate",
                "country": "ES",
                "city": "Madrid",
                "region": "Madrid",
            },
        }
    ],
    response_model=Places,
)


In [ ]:
pprint(response.to_dict())

In [ ]:
pprint(completion)

In [ ]:
pprint(response.citations)

In [ ]:
response.summary

In [ ]:
completion.output[1].content[0]

In [ ]:
idx = [(ann.start_index, ann.end_index) for ann in completion.output[1].content[0].annotations]
print(idx)
response.summary[idx[0][0] : idx[0][1]]
